In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import openslide as ops
import sys,stat
import pydot
from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from fastprogress import progress_bar
import copy
from collections import namedtuple
import os
import random
import shutil
import time
from PIL import Image,ImageDraw,ImageEnhance,ImageColor 

import pandas as pd
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt


Image.MAX_IMAGE_PIXELS = 10000000000

In [2]:
data_types = ['gc0','gc1','gcpp0', 'gcpp1']
for model_path in ['Xception_bad1','Xception_bad2','ResNet152V2_bad2','ResNet152V2_bad1','vgg16_bad1', 'vgg16_bad2']:
    
#     for types,mask_id in progress_bar([(1,1),(1,3),(1,4),(4,1),(4,2),(4,3),(4,4),(4,5),(7,1),(7,4)]):
    for types,mask_id in progress_bar([(7,4)]):    
        for i in range(0,len(data_types)):
            mask_size = Image.open('./result/' + str(model_path) + '/gc0_'   + str(types) + '_' + str(mask_id) + '.png').size
            god_img_mask = copy.deepcopy(np.asarray(Image.open(r'./train/test/' + str(types) + '/svs/' + str(types) + '_' + str(mask_id) + r'_good.png')))
            god_img_mask = cv2.resize(god_img_mask, (mask_size[0], mask_size[1]), interpolation = cv2.INTER_AREA)
            
            
            
            
            data_type = data_types[i]
            test_mask = np.asarray(Image.open('./result/' + str(model_path) + '/' + data_type + '_'   + str(types) + '_' + str(mask_id) + '.png'))[...,1]

    

In [3]:
for types,mask_id in progress_bar([(7,4)]):   
    mask_size = Image.open('./result/' + str(model_path) + '/gc0_'   + str(types) + '_' + str(mask_id) + '.png').size
    god_img_mask = copy.deepcopy(np.asarray(Image.open(r'./train/test/' + str(types) + '/svs/' + str(types) + '_' + str(mask_id) + r'_good.png')))
    god_img_mask = cv2.resize(god_img_mask, (mask_size[0], mask_size[1]), interpolation = cv2.INTER_AREA)

    bad_img_mask = copy.deepcopy(np.asarray(Image.open(r'./train/test/' + str(types) + '/svs/' + str(types) + '_' + str(mask_id) + r'_bad.png')))
    bad_img_mask = cv2.resize(bad_img_mask, (mask_size[0], mask_size[1]), interpolation = cv2.INTER_AREA)

    tif = ops.open_slide(r'./train/test/' + str(types) + '/svs/' + str(types) + '_' + str(mask_id) + r'.svs')
    total_size = tif.dimensions
    tif_image = np.asarray(tif.read_region((0,0),0,(total_size[0],total_size[1])))[:,:,:3]
    tif_image = cv2.resize(tif_image, (mask_size[0], mask_size[1]), interpolation = cv2.INTER_AREA)

In [29]:
# for model_path in ['Xception_bad1','Xception_bad2','ResNet152V2_bad2','ResNet152V2_bad1','vgg16_bad1', 'vgg16_bad2']:


# for model_path in ['Xception_bad1','Xception_bad2','ResNet152V2_bad2','ResNet152V2_bad1','vgg16_bad1', 'vgg16_bad2']:
for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))


        

In [30]:
lenth = 3000
y0,x0 = 2409,6685
y1,x1 = y0+lenth,x0+lenth

x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2


In [31]:
image_id = '5_4'

In [32]:

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [11]:
Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )

In [ ]:
3000像素：

1：
21985,2878
36997,17610
6750,10603
34952,26324

2：
6947,6154
4441,10451

3：
12820,37566
23077,46507
28134,39857
19201,35920
36061,3694

4：
2136,8333
3636,5231
1820,4208
7046,5124

5：
2296,12626
5634,13210
7907,7904
2409,6685

6：
6239,2962
3811,1978
6264,5999
1435,9589

7：
14759,6382
14836,12000
18690,8265
24166,15439
16000,9054

8：
16751,7650
13054,7517
4565,2591
8828,16515
10845,24806

In [ ]:
3607,1035
33137,19809
45297,17337

In [ ]:
6557,5860
4327,1165
1069,8878

In [ ]:
10712,33812
25797,39754
41199,7070

In [ ]:
3296,1625
1243,8008
5876,3987

In [ ]:
2181,6629
5087,3399
3057,10004

In [ ]:
3252,1642
1,6703

In [ ]:
812,6300
21111,12171
16468,5444

In [ ]:
1309,17021
10268,16564
15230,5564

In [ ]:
(1,1),(1,3),(1,4),(4,1),(4,2),(4,5),(7,1),(7,4)
  1     2    3     4      5     6     7     8 

In [4]:
id1 = '8_1'
id2 = '8_2'
id3 = '8_3'
b0,a0 = 1309,17021
b1,a1 = 10268,16564
b2,a2 = 15230,5564

In [5]:
image_id = id1
lenth = 6000
y0,x0 = b0,a0
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [6]:
image_id = id2
lenth = 6000
y0,x0 = b1,a1
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [7]:
image_id = id3
lenth = 6000
y0,x0 = b2,a2
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/6000/vgg/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/6000/vgg/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [4]:
image_id = '8_1'
lenth = 3000
y0,x0 = 16751,7650
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [5]:
image_id = '8_2'
lenth = 3000
y0,x0 = 13054,7517
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [6]:
image_id = '8_3'
lenth = 3000
y0,x0 = 4565,2591
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [9]:
image_id = '8_4'
lenth = 3000
y0,x0 = 8828,16515
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [10]:
image_id = '8_5'
lenth = 3000
y0,x0 = 10845,24806
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['vgg16_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['vgg16_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [4]:
image_id = '8_1'
lenth = 3000
y0,x0 = 16751,7650
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [5]:
image_id = '8_2'
lenth = 3000
y0,x0 = 13054,7517
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [6]:
image_id = '8_3'
lenth = 3000
y0,x0 = 4565,2591
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [7]:
image_id = '8_4'
lenth = 3000
y0,x0 = 8828,16515
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

In [8]:
image_id = '8_5'
lenth = 3000
y0,x0 = 10845,24806
y1,x1 = y0+lenth,x0+lenth
x0,x1,y0,y1 = x0//2, x1//2, y0//2, y1//2

for model_path in ['ResNet152V2_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['ResNet152V2_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/res/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/res/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad1']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )

for model_path in ['Xception_bad2']:
    
    gc0_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_' + str(types) + '_' + str(mask_id) + '.png'))
    gc1_heatmap    = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'+ str(types) + '_' + str(mask_id) + '.png'))
    gcpp0_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '.png'))
    gcpp1_heatmap  = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '.png'))

    gc0_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gc1_smooth_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp0_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))
    gcpp1_smooth_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_smooth_mask.png'))

    gc0_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc0_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gc1_mask     = np.asarray(Image.open('./result/' + str(model_path) + '/gc1_'  + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp0_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp0_' + str(types) + '_' + str(mask_id) + '_mask.png'))
    gcpp1_mask   = np.asarray(Image.open('./result/' + str(model_path) + '/gcpp1_' + str(types) + '_' + str(mask_id) + '_mask.png'))

Image.fromarray(cv2.resize((god_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_goodmask.png'  )
Image.fromarray(cv2.resize((bad_img_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + str(image_id) + '_badmask.png'  )
Image.fromarray(cv2.resize((tif_image[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_tif.png'  )

# Image.fromarray(gc0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gc1_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp0_heatmap[y0:y1, x0:x1]).save('./')
# Image.fromarray(gcpp1_heatmap[y0:y1, x0:x1]).save('./')
Image.fromarray(cv2.resize((gc0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gc1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gc1_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp0_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp0_' + str(model_path) + '.png'  )
Image.fromarray(cv2.resize((gcpp1_heatmap[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).save('./结果制图/xce/' + str(image_id) + '_gcpp1_' + str(model_path) + '.png'  )

Image.fromarray(cv2.resize((gc0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gc1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_mask.png'  )

Image.fromarray(cv2.resize((gc0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gc1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gc1_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp0_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp0_' + str(model_path) + '_smooth_mask.png'  )
Image.fromarray(cv2.resize((gcpp1_smooth_mask[y0:y1, x0:x1]),(512,512),interpolation = cv2.INTER_AREA)).convert('L').save('./结果制图/xce/' + '/' + str(image_id) + '_gcpp1_' + str(model_path) + '_smooth_mask.png'  )